In [1]:
import sys
sys.path.append('/Projects/regionintelligenceai/map/')

In [2]:
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [3]:
# Import driver configuration
import time
import numpy as np
from src.driver_config import get_chrome_driver, navigate_and_print_title
from src.const import FULLERTON_PLANNING_URL
import pandas as pd
from src.paths import *
from datetime import datetime
driver = get_chrome_driver()

URL = FULLERTON_PLANNING_URL

# Navigate to the URL and print its title
navigate_and_print_title(driver, URL)

# If you need to close the driver after use (recommended if not using further in the notebook):
driver.quit()

Development Activity | Fullerton, CA


In [ ]:
class FullertonScraper:
    # Define current_date as a class variable
    current_date = datetime.now().strftime('%Y-%m-%d')

    def __init__(self, driver):
        self.driver = driver
        self.heading_name = []
        self.listing_names = []
        self.case_number_texts = []
        self.project_locations = []
        self.type_of_use = []
        self.planner_leads = []
        self.project_descriptions = []
        self.project_status = []
        self.image_urls = []

    def connect(self, url):
        self.driver.get(url)
        print(self.driver.title)

    def scrape_data(self):
        try:
            main = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "govAccess-reTable-4"))
            )

            headers = main.find_elements(By.CSS_SELECTOR, "tr:first-of-type td")
            for header in headers:
                self.heading_name.append(header.text)

            types = main.find_elements(By.CSS_SELECTOR, "tr:nth-of-type(n+2)")
            for type in types:
                first_td = type.find_element(By.CSS_SELECTOR, "td:first-of-type")
                self.type_of_use.append(first_td.text)

            listings = main.find_elements(By.CSS_SELECTOR, "tr:nth-of-type(n+2)")
            for listing in listings:
                second_td = listing.find_element(By.CSS_SELECTOR, "td:nth-of-type(2)")
                self.listing_names.append(second_td.text)

                third_td = listing.find_element(By.CSS_SELECTOR, "td:nth-of-type(3)")
                self.project_locations.append(third_td.text)

                fourth_td = listing.find_element(By.CSS_SELECTOR, "td:nth-of-type(4)")
                self.project_status.append(fourth_td.text)

                fifth_td = listing.find_element(By.CSS_SELECTOR, "td:nth-of-type(5)")
                self.case_number_texts.append(fifth_td.text)

                sixth_td = listing.find_element(By.CSS_SELECTOR, "td:nth-of-type(6)")
                self.planner_leads.append(sixth_td.text)

        except Exception as e:
            print(f"An error occurred: {e}")

    def create_dataframe(self):
        listing_names = [item.split("\n")[0] for item in self.listing_names]
        description = [item.split("\n")[1] for item in self.listing_names]

        df = pd.DataFrame(
            {
                "projectName": listing_names,
                "address": self.project_locations,
                "planner": self.planner_leads,
                "status": self.project_status,
                #"Type of Use": self.type_of_use,
                "description": description
            }
        )
        # add phone, email, recent
        return df
    
    def save_to_raw(self, path=None):
        if not path:
            path = RAW_DATA_DIR / 'fullerton' / f'fullerton_city_data_{FullertonScraper.current_date}.csv'
        df = self.create_dataframe()
        df.to_csv(path, index=False)

        

def main_city_of_fullerton():
    driver = get_chrome_driver
    scraper = FullertonScraper(driver)
    scraper.connect(FULLERTON_PLANNING_URL)
    scraper.scrape_data()
    df = scraper.create_dataframe()
    driver.close()
    return df


In [8]:
from datetime import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from src.driver_config import get_chrome_driver
from src.const import fullerton_planner_names, fullerton_planner_phones
from src.const import FULLERTON_PLANNING_URL, FULLERTON_PLANNING_OFFICE_EMAIL
from src.paths import RAW_DATA_DIR
from difflib import SequenceMatcher

class FullertonScraper:

    # Define current_date as a class variable
    current_date = datetime.now().strftime('%Y-%m-%d')

    def __init__(self, driver):
        self.driver = driver
        self.heading_name = []
        self.listing_names = []
        self.case_number_texts = []
        self.project_locations = []
        self.type_of_use = []
        self.planner_leads = []
        self.project_descriptions = []
        self.project_status = []
        self.image_urls = []

    def connect(self, url):
        self.driver.get(url)
        print(self.driver.title)

    def scrape_data(self):
        try:
            main = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "govAccess-reTable-4"))
            )

            headers = main.find_elements(By.CSS_SELECTOR, "tr:first-of-type td")
            for header in headers:
                self.heading_name.append(header.text)

            types = main.find_elements(By.CSS_SELECTOR, "tr:nth-of-type(n+2)")
            for type in types:
                first_td = type.find_element(By.CSS_SELECTOR, "td:first-of-type")
                self.type_of_use.append(first_td.text)

            listings = main.find_elements(By.CSS_SELECTOR, "tr:nth-of-type(n+2)")
            for listing in listings:
                second_td = listing.find_element(By.CSS_SELECTOR, "td:nth-of-type(2)")
                self.listing_names.append(second_td.text)

                third_td = listing.find_element(By.CSS_SELECTOR, "td:nth-of-type(3)")
                self.project_locations.append(third_td.text)

                fourth_td = listing.find_element(By.CSS_SELECTOR, "td:nth-of-type(4)")
                self.project_status.append(fourth_td.text)

                fifth_td = listing.find_element(By.CSS_SELECTOR, "td:nth-of-type(5)")
                self.case_number_texts.append(fifth_td.text)

                sixth_td = listing.find_element(By.CSS_SELECTOR, "td:nth-of-type(6)")
                self.planner_leads.append(sixth_td.text)

        except Exception as e:
            print(f"An error occurred: {e}")

    def create_dataframe(self):
        listing_names = [item.split("\n")[0] for item in self.listing_names]
        description = [item.split("\n")[1] if '\n' in item else '' for item in self.listing_names]

        df = pd.DataFrame(
            {
                "projectName": listing_names,
                "address": self.project_locations,
                "planner": self.planner_leads,
                "status": self.project_status,
                "type_of_use": self.type_of_use,
                "description": description
            }
        )

        df['planner'] = df['planner'].str.strip()
        
        # Helper function to get the closest match from the dictionary
        def get_closest_match(name):
            max_ratio = 0
            best_match = None
            for key, value in fullerton_planner_names.items():
                ratio = SequenceMatcher(None, name, value).ratio()
                if ratio > max_ratio:
                    max_ratio = ratio
                    best_match = value
        
        # If similarity is more than 30%, return the matched name, otherwise return the original name
            return best_match if max_ratio > 0.3 else name

        df["planner"] = df["planner"].apply(get_closest_match)
    
    # Assuming you have another dictionary called fullerton_planner_phones which maps names to phone numbers
        df["phone"] = df["planner"].map(fullerton_planner_phones)

    # Replace '\n' with ' ' in the 'status' column
        df["status"] = df["status"].str.replace('\n', ' ')

        df['city'] = 'Fullerton'

        df['email'] = FULLERTON_PLANNING_OFFICE_EMAIL

        # Add recentUpdate column with the first day of the current month for every row
        df['recentUpdate'] = datetime.now().replace(day=1).strftime('%Y-%m-%d')
        
        return df
    
    def save_to_raw(self, path=None):
        if not path:
            path = RAW_DATA_DIR / 'fullerton' / f'fullerton_city_data_{FullertonScraper.current_date}.xlsx'
        df = self.create_dataframe()
        df.to_csv(path, index=False)

def main_city_of_fullerton():
    driver = get_chrome_driver()
    scraper = FullertonScraper(driver)
    scraper.connect(FULLERTON_PLANNING_URL)
    scraper.scrape_data()
    df = scraper.create_dataframe()
    scraper.save_to_raw()
    driver.quit()
    return df

if __name__ == "__main__":
    df = main_city_of_fullerton()
    print(df)


Development Activity | Fullerton, CA
                                projectName  \
0                             Pointe Common   
1                  245 N State College Blvd   
2                          Parkwest Project   
3              Rexford Industrial Fullerton   
4              Rexford Via Burton Fullerton   
5    Acacia and Kimberly Industrial Project   
6                             The Fox Block   
7                         Goodman Logistics   
8          Casa Bella - Highland & Valencia   
9                             Hub Fullerton   
10                   Streetlights Fullerton   
11             The Pines at Sunrise Village   
12              East Amerige Avenue Project   
13                         Pathways of Hope   
14                       Highland Tract Map   
15          Brookhurst/Orangethorpe Project   
16                      Shoe City Billboard   
17                       Truck Yard Project   
18                           76 Gas Station   
19                     

In [6]:
df.planner.unique()

array([' Edgardo Caldera Senior Planner',
       ' Bradley Misner Contract Planner',
       'Edgardo Caldera Senior Planner',
       ' Andrew Kusch Associate Planner',
       ' Chris Schaefer Planning Manager'], dtype=object)

In [9]:
df

,projectName,address,planner,status,type_of_use,description,phone
0,Pointe Common,1600 W. Commonwealth,Edgardo Caldera,Final CEQA ISMND Approved by City Council on A...,Residential,"62 Unit Multi-Family Residential Units, All Un...",(714)773-5773
1,245 N State College Blvd,245 N State College Blvd,Edgardo Caldera,"Approved by Zoning Administrator on April 6th,...",Residential,Request to Construct 25 Residential Town Homes...,(714)773-5773
2,Parkwest Project,3 Parcels - 2 Parcels South of East Santa Fe &...,Matt Corrigan,"Council Approved Land Sale on December 20, 202...",Mixed Use,"Proposed 124 room hotel, 140 residential units...",(714)773-5799
3,Rexford Industrial Fullerton,1500 S. Raymond,Edgardo Caldera,"3rd Review Complete -DCRC March 1, 2023 Notice...",Industrial Development,(Hotel Fullerton) - Demolition of hotel and co...,(714)773-5773
4,Rexford Via Burton Fullerton,1901 Via Burton\n,Edgardo Caldera,Entitlements Complete Currently Undergoing Con...,Industrial Development,Major Site Plan to demolish an existing indust...,(714)773-5773
5,Acacia and Kimberly Industrial Project,801 Acacia Ave,Edgardo Caldera,Preliminary Review DCRC Comments Provided on ...,Industrial Development,"Request for a Major Site Plan, pursuant to Ful...",(714)773-5773
6,The Fox Block,Harbor Boulevard and Chapman Avenue,Matt Corrigan,Entitlements Complete City Council Approved La...,Commercial Development,Proposed Public Parking Structure and Commerci...,(714)773-5799
7,Goodman Logistics,2001 E. Orangethorpe Avenue,Edgardo Caldera,Entitlements Complete Construction Near Comple...,New Industrial Development,"Goodman Logistics Center, Fullerton- Construct...",(714)773-5773
8,Casa Bella - Highland & Valencia,415 S. Highland Ave.,Edgardo Caldera,Entitlements Complete Grading Permits have bee...,Mixed-Use,The project proposes to construct 20 residenti...,(714)773-5773
9,Hub Fullerton,"2601, 2701, and 2751 E. Chapman Avenue",Edgardo Caldera,Entitlements Complete Currently proceeding wit...,Student-Oriented Housing,Student oriented housing - 420 units Additiona...,(714)773-5773
